In [1]:
import gensim
from gensim import corpora, models, similarities
import pickle
import multiprocessing
import numpy as np
import pandas as pd
import logging
import warnings

# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
warnings.filterwarnings(action='ignore')

assert gensim.models.doc2vec.FAST_VERSION > -1
cores = multiprocessing.cpu_count()

# unlabelled_corpus = pickle.load(open('./input/unlabelled_corpus_clean3.p','rb'))
true_corpus = pickle.load(open('./input/true_corpus_clean3.p','rb'))
fake_corpus = pickle.load(open('./input/fake_corpus_clean3.p','rb'))

C:\Anaconda\envs\py36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
dictionary = corpora.Dictionary(true_corpus + fake_corpus)
# unlabelled_bow = [dictionary.doc2bow(s) for s in unlabelled_corpus]
true_bow = [dictionary.doc2bow(s) for s in true_corpus]
fake_bow = [dictionary.doc2bow(s) for s in fake_corpus]

In [3]:
num_topics = 100
chunksize = 400
passes = 5

hasNext = True
start = 0
f = 1000
ctr = 0

curr_bow = true_bow + fake_bow
model = models.LdaModel(curr_bow, id2word=dictionary, num_topics=num_topics,alpha = 'auto',eta='auto',random_state=0, chunksize=chunksize, passes=passes)
# model = pickle.load(open('./input/lda_model.p','rb'))

In [4]:
while(hasNext):  
    s = (ctr * f)+start
    e = min(s + f, len(curr_bow))
    print ('REUPDATE:'+ str(s))
    if (s + f >= len(curr_bow)):
        hasNext = False
    model.update(curr_bow[s:e])
    ctr +=1
    pickle.dump(model,open('./input/lda_model.p','wb'))

REUPDATE:472000
REUPDATE:473000
REUPDATE:474000
REUPDATE:475000
REUPDATE:476000
REUPDATE:477000
REUPDATE:478000
REUPDATE:479000
REUPDATE:480000
REUPDATE:481000
REUPDATE:482000
REUPDATE:483000
REUPDATE:484000
REUPDATE:485000
REUPDATE:486000
REUPDATE:487000
REUPDATE:488000
REUPDATE:489000
REUPDATE:490000
REUPDATE:491000
REUPDATE:492000
REUPDATE:493000
REUPDATE:494000
REUPDATE:495000
REUPDATE:496000
REUPDATE:497000
REUPDATE:498000
REUPDATE:499000
REUPDATE:500000
REUPDATE:501000
REUPDATE:502000
REUPDATE:503000
REUPDATE:504000
REUPDATE:505000
REUPDATE:506000
REUPDATE:507000
REUPDATE:508000
REUPDATE:509000
REUPDATE:510000
REUPDATE:511000
REUPDATE:512000
REUPDATE:513000
REUPDATE:514000


In [4]:
def gen_frame(corpus_bow,model,num_topics):
    topic_dists = np.zeros([len(corpus_bow),num_topics])
    
    for i,item in enumerate(corpus_bow):       
        dists = model.get_document_topics(item)        
        indices = list(dict(dists).keys())        
        vals = list(dict(dists).values())        
        topic_dists[i,indices] = vals

        topic_dists = pd.DataFrame(topic_dists, columns = ['topic'+str(a) , for a in range(num_topics)])
    return topic_dists

In [5]:
unlabelled_lda_vec = gen_frame(unlabelled_bow, model, num_topics)
true_lda_vec = gen_frame(true_bow, model, num_topics)
fake_lda_vec = gen_frame(fake_bow, model, num_topics)

In [5]:
# pickle.dump(unlabelled_lda_vec, open('./input/unlabelled_lda_vec_sm.p','wb'))
# pickle.dump(true_lda_vec, open('./input/true_lda_vec_sm.p','wb'))
# pickle.dump(fake_lda_vec, open('./input/fake_lda_vec_sm.p','wb'))
pickle.dump(dictionary, open('./input/corpus_dict.p','wb'))
pickle.dump(model, open('./input/lda_model.p','wb'))

In [ ]:
len(unlabelled_bow)